In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.datasets.dataset_metadata import DatasetMetadata
from torchsig.signals.builders.constellation import ConstellationSignalBuilder
import torchsig.transforms.functional as F
from torchsig.utils.dsp import (
    multistage_polyphase_resampler
)

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def generate_qpsk_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband QPSK Signal.

        Args:
        num_iq_samples (int, optional): Length of sample. Defaults to 10.
        scale (int, optional): scale normalized signal data. Defaults to 1.0.

        Returns:
            signal: generated Signal 

    """
    sample_rate = 10e6
    md = DatasetMetadata(
        num_iq_samples_dataset = num_iq_samples,
        fft_size = 4,
        impairment_level = 0,
        sample_rate = sample_rate,
        num_signals_max = 1,
        num_signals_min = 1,
        num_signals_distribution = [1.0],
        snr_db_min = 100.0,
        snr_db_max = 100.0,
        signal_duration_min = 1.00*num_iq_samples/sample_rate,
        signal_duration_max = 1.00*num_iq_samples/sample_rate,
        signal_bandwidth_min = sample_rate/16,
        signal_bandwidth_max = sample_rate/16,
        signal_center_freq_min = 0.0,
        signal_center_freq_max = 0.0,         
        class_list = ['qpsk'],
        class_distribution = [1.0],
        seed = 42
    )

    builder = ConstellationSignalBuilder(
        dataset_metadata = md, 
        class_name = 'qpsk',
        seed = 42
    )
    signal = builder.build()

    # normalize, then scale data   
    signal.data = F.normalize(
        data = signal.data,
        norm_order = 2,
        flatten = False
    )
    signal.data = np.multiply(signal.data, scale)

    return signal

In [ ]:
# test data
N = 4096
qpsk_bb_data = generate_qpsk_signal(num_iq_samples = N, scale = 1.0).data

# upsample and rescale
qpsk_8x_data = multistage_polyphase_resampler(qpsk_bb_data, 8.0)
qpsk = qpsk_8x_data * np.sqrt(N) # set qpsk mean_power to 1.0 W
qpsk_mean_power = np.mean(np.abs(qpsk)**2)
print(qpsk_mean_power)

In [ ]:
# examine transform response across qpsk input powers
pin_dB = np.arange(-30, 10+1, 1) # [-30, +10] dB input power range
pin = 10 ** (pin_dB / 10) # input power, linear units

gain = 1.0       # linear
psat = 1.0       # dB
phi_max = -0.1   # rad
phi_slope = 0.1  # linear power/rad

pow_out = np.zeros((len(pin),))
ph_out = np.zeros((len(pin),))

for i,p in enumerate(pin):
    qpsk_in = qpsk * np.sqrt(pin[i] / 1.0)              # set qpsk power (original 1.0 linear power)
    psat_backoff =  psat / np.mean(np.abs(qpsk_in)**2)  # set constant operating point
    response_data = F.nonlinear_amplifier(
        data = qpsk_in,
        gain = gain,
        psat_backoff = psat_backoff,
        phi_max = phi_max,
        phi_slope = phi_slope,
        auto_scale = False
    )
    pow_out[i] = np.mean(np.abs(response_data)**2)      # mean power output estimate
    ph_out[i] = np.mean(np.angle(response_data) - np.angle(qpsk_in)) # relative phase shift estimate

    if (pin_dB[i] == -10 or pin_dB[i] == -5 or pin_dB[i] == 0):
        # generate a plot
        fig = plt.figure(figsize=(12,3))
        ax = fig.add_subplot(1,2,1)
        ax.plot(np.real(qpsk_in),np.imag(qpsk_in),label='in')
        ax.plot(np.real(response_data),np.imag(response_data),label='out')
        ax.set_aspect('equal')
        ax.set_title('Pin dB = ' + str(pin_dB[i]) + ' dB')
        ax.legend(loc='upper right')
        ax.grid()
        ax = fig.add_subplot(1,2,2)
        ax.plot(np.real(qpsk_in),label='in')
        ax.plot(np.real(response_data),label='out')
        ax.legend(loc='upper right')
        ax.grid()

pin_log10 = 10*np.log10(np.where(pin == 0, float('inf'), pin))
pout_log10 = 10*np.log10(np.where(pow_out == 0, float('inf'), pow_out))

# plot pin/pout
fig = plt.figure(figsize=(8, 8))
plt.plot(pin_log10, pout_log10)
plt.title('Nonlinear Amplifier Model (AM/AM)',fontsize='large');
plt.xlabel('Pin (dB)',fontsize='large');
plt.ylabel('Pout (dB)',fontsize='large');
plt.show()  

# plot pin/ph_out
fig = plt.figure(figsize=(8, 8))
plt.plot(pin_log10, ph_out)
plt.title('Nonlinear Amplifier Model (AM/PM)',fontsize='large');
plt.xlabel('Pin (dB)',fontsize='large');
plt.ylabel('$\\phi$ (rad)',fontsize='large');
plt.show()
